Predicting Restaurant Food Cost 


Who doesn’t love food? All of us must have craving for at least a few favourite food items, we may also have a few places where we like to get them, a restaurant which serves our favourite food the way we want it to be. But there is one factor that will make us reconsider having our favourite food from our favourite restaurant, the cost. Here in this hackathon, you will be predicting the cost of the food served by the restaurants across different cities in India. You will use your Data Science skills to investigate the factors that really affect the cost, and who knows maybe you will even gain some very interesting insights that might help you choose what to eat and from where.

Size of training set: 12,690 records

Size of test set: 4,231 records



Size of training set: 12,690 records

Size of test set: 4,231 records

FEATURES:
TITLE: The feature of the restaurant which can help identify what and for whom it is suitable for.

RESTAURANT_ID: A unique ID for each restaurant.

CUISINES: The variety of cuisines that the restaurant offers.

TIME: The open hours of the restaurant.

CITY: The city in which the restaurant is located.

LOCALITY: The locality of the restaurant.

RATING: The average rating of the restaurant by customers.

VOTES: The overall votes received by the restaurant.

COST: The average cost of a two-person meal.

In [1]:
#import the libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tabulate import tabulate
from sklearn import metrics
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from scipy.stats import skew
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline


In [5]:

foodData= pd.read_excel('Food_Data.xlsx') 
foodData

,TITLE,RESTAURANT_ID,CUISINES,TIME,CITY,LOCALITY,RATING,VOTES,COST
0,CASUAL DINING,9438,"Malwani, Goan, North Indian","11am – 4pm, 7:30pm – 11:30pm (Mon-Sun)",Thane,Dombivali East,3.6,49 votes,1200
1,"CASUAL DINING,BAR",13198,"Asian, Modern Indian, Japanese",6pm – 11pm (Mon-Sun),Chennai,Ramapuram,4.2,30 votes,1500
2,CASUAL DINING,10915,"North Indian, Chinese, Biryani, Hyderabadi","11am – 3:30pm, 7pm – 11pm (Mon-Sun)",Chennai,Saligramam,3.8,221 votes,800
3,QUICK BITES,6346,"Tibetan, Chinese",11:30am – 1am (Mon-Sun),Mumbai,Bandra West,4.1,24 votes,800
4,DESSERT PARLOR,15387,Desserts,11am – 1am (Mon-Sun),Mumbai,Lower Parel,3.8,165 votes,300
...,...,...,...,...,...,...,...,...,...
12685,QUICK BITES,13228,"North Indian, Burger, Kebab","12noon – 12midnight (Mon, Tue, Wed, Thu, Sun)...",Hyderabad,Gachibowli,3.8,546 votes,500
12686,"CASUAL DINING,BAR",9686,"Goan, Continental","12noon – 1am (Mon-Fri),11am – 5pm, 7pm – 1am...",Mumbai,Bandra Kurla Complex,4.3,1214 votes,1800
12687,LOUNGE,11133,"Finger Food, Continental, Asian, Chinese",12noon – 12:30AM (Mon-Sun),Navi Mumbai,Vashi,4.0,608 votes,1300
12688,CASUAL DINING,6134,"North Indian, South Indian, Chinese, Street Food",6am – 10:45pm (Mon-Sun),Chennai,Maduravoyal,3.5,32 votes,400


In [6]:
foodData.isnull().sum()

TITLE               0
RESTAURANT_ID       0
CUISINES            0
TIME                0
CITY              112
LOCALITY           98
RATING              2
VOTES            1204
COST                0
dtype: int64

In [16]:
foodData['RATING'].unique()

array(['3.6', '4.2', '3.8', '4.1', '4.0', '4.3', '3.9', '3.3', '3.4', '',
       '4.5', '3.5', '4.4', '2.7', '3.7', '4.7', 'NEW', '3.1', '2.5',
       '4.6', '2.8', nan, '3.0', '3.2', '2.6', '2.9', '4.9', '4.8', '2.4',
       '2.3', '2.0', '2.1', '2.2'], dtype=object)

In [17]:
foodData['VOTES'].unique()

array(['49 ', '30 ', '221 ', ..., '2723 ', '1426 ', '1214 '], dtype=object)

In [24]:
# replace the values
foodData['VOTES']=foodData['VOTES'].str.replace('votes', '')
foodData['RATING']=foodData['RATING'].str.replace('-', '')
foodData['RATING']=foodData['RATING'].str.replace('NEW', '0')
foodData['RATING']=foodData['RATING'].str.replace('', '0')

In [27]:
foodData['RATING']=foodData['RATING'].astype(float)

In [28]:
foodData['RATING'].unique()

array([30.06, 40.02, 30.08, 40.01, 40.  , 40.03, 30.09, 30.03, 30.04,
        0.  , 40.05, 30.05, 40.04, 20.07, 30.07, 40.07, 30.01, 20.05,
       40.06, 20.08,   nan, 30.  , 30.02, 20.06, 20.09, 40.09, 40.08,
       20.04, 20.03, 20.  , 20.01, 20.02])

In [29]:
foodData['RATING']=foodData['RATING'].fillna(foodData['RATING'].astype(float).mean() )
foodData['VOTES']=foodData['VOTES'].fillna(foodData['VOTES'].astype(float).mean() )


In [31]:
#For categorical features  

foodData['CITY']= foodData.apply(lambda x: foodData['CITY'].fillna(foodData['CITY'].value_counts().index[0]))

foodData['LOCALITY']= foodData.apply(lambda x: foodData['LOCALITY'].fillna(foodData['LOCALITY'].value_counts().index[0]))

In [30]:
foodData.isnull().sum()

TITLE              0
RESTAURANT_ID      0
CUISINES           0
TIME               0
CITY             112
LOCALITY          98
RATING             0
VOTES              0
COST               0
dtype: int64

In [32]:
foodData

,TITLE,RESTAURANT_ID,CUISINES,TIME,CITY,LOCALITY,RATING,VOTES,COST
0,CASUAL DINING,9438,"Malwani, Goan, North Indian","11am – 4pm, 7:30pm – 11:30pm (Mon-Sun)",Thane,Dombivali East,30.06,49,1200
1,"CASUAL DINING,BAR",13198,"Asian, Modern Indian, Japanese",6pm – 11pm (Mon-Sun),Chennai,Ramapuram,40.02,30,1500
2,CASUAL DINING,10915,"North Indian, Chinese, Biryani, Hyderabadi","11am – 3:30pm, 7pm – 11pm (Mon-Sun)",Chennai,Saligramam,30.08,221,800
3,QUICK BITES,6346,"Tibetan, Chinese",11:30am – 1am (Mon-Sun),Mumbai,Bandra West,40.01,24,800
4,DESSERT PARLOR,15387,Desserts,11am – 1am (Mon-Sun),Mumbai,Lower Parel,30.08,165,300
...,...,...,...,...,...,...,...,...,...
12685,QUICK BITES,13228,"North Indian, Burger, Kebab","12noon – 12midnight (Mon, Tue, Wed, Thu, Sun)...",Hyderabad,Gachibowli,30.08,546,500
12686,"CASUAL DINING,BAR",9686,"Goan, Continental","12noon – 1am (Mon-Fri),11am – 5pm, 7pm – 1am...",Mumbai,Bandra Kurla Complex,40.03,1214,1800
12687,LOUNGE,11133,"Finger Food, Continental, Asian, Chinese",12noon – 12:30AM (Mon-Sun),Navi Mumbai,Vashi,40.00,608,1300
12688,CASUAL DINING,6134,"North Indian, South Indian, Chinese, Street Food",6am – 10:45pm (Mon-Sun),Chennai,Maduravoyal,30.05,32,400


In [33]:
# Applying LabelEncoder  

from sklearn.preprocessing import LabelEncoder
transcol=['TITLE','CITY','LOCALITY']
for col in foodData :
    
    for i in transcol:
        
        if col==i  :
            print(i)
            labelencoder = LabelEncoder()
            foodData[col] = labelencoder.fit_transform(foodData[col])

TITLE
CITY
LOCALITY


In [34]:
foodData

,TITLE,RESTAURANT_ID,CUISINES,TIME,CITY,LOCALITY,RATING,VOTES,COST
0,31,9438,"Malwani, Goan, North Indian","11am – 4pm, 7:30pm – 11:30pm (Mon-Sun)",320,285,30.06,49,1200
1,33,13198,"Asian, Modern Indian, Japanese",6pm – 11pm (Mon-Sun),67,1046,40.02,30,1500
2,31,10915,"North Indian, Chinese, Biryani, Hyderabadi","11am – 3:30pm, 7pm – 11pm (Mon-Sun)",67,1101,30.08,221,800
3,95,6346,"Tibetan, Chinese",11:30am – 1am (Mon-Sun),222,136,40.01,24,800
4,50,15387,Desserts,11am – 1am (Mon-Sun),222,620,30.08,165,300
...,...,...,...,...,...,...,...,...,...
12685,95,13228,"North Indian, Burger, Kebab","12noon – 12midnight (Mon, Tue, Wed, Thu, Sun)...",131,342,30.08,546,500
12686,33,9686,"Goan, Continental","12noon – 1am (Mon-Fri),11am – 5pm, 7pm – 1am...",222,133,40.03,1214,1800
12687,71,11133,"Finger Food, Continental, Asian, Chinese",12noon – 12:30AM (Mon-Sun),239,1342,40.00,608,1300
12688,31,6134,"North Indian, South Indian, Chinese, Street Food",6am – 10:45pm (Mon-Sun),67,631,30.05,32,400


In [39]:
foodData.drop(['RESTAURANT_ID','CUISINES','TIME','LOCALITY'],axis=1,inplace=True)
# DoctorsData.drop(['Place'],axis=1,inplace=True)

In [42]:

# IQR score technique to remove Outliers 
Q1 = foodData.quantile(0.25)
Q3 = foodData.quantile(0.75)
IQR = Q3 - Q1
print(IQR.shape)
dfIQR = foodData[~((foodData < (Q1-1.5 * IQR)) |(foodData > (Q3 + 1.5 * IQR))).any(axis=1)]
dfIQR.shape


(4,)


(10641, 5)

In [43]:
x=dfIQR.drop(['COST'],axis=1)
x.shape

(10641, 4)

In [45]:
y=dfIQR['COST']
y=np.array(y).reshape(-1,1)

In [46]:
x.skew()

TITLE     -0.068489
CITY       0.361283
RATING     0.029339
VOTES     17.574296
dtype: float64

In [48]:
#Apply Log Transformation
x['VOTES']=np.log10(x['VOTES'].astype(int)+1)

In [50]:
x.skew()

TITLE    -0.068489
CITY      0.361283
RATING    0.029339
VOTES     0.014375
dtype: float64

In [51]:
# splitting data as X_train and X_test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2,random_state = 42)

In [52]:
#Linear Regression
regressor = LinearRegression()  
regressor.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [53]:
y_pred = regressor.predict(X_test)

In [54]:
# calculating RMSE
from sklearn.metrics import mean_squared_error
from math import sqrt
rmse = sqrt(mean_squared_error(y_test, y_pred))
rmse

278.24937500923573

In [55]:
df = pd.DataFrame({'Actual': np.array(y_test)[:,0], 'Predicted': y_pred[:,0]})
df

,Actual,Predicted
0,600,545.547123
1,150,591.836103
2,600,597.134990
3,600,619.654017
4,600,514.795459
...,...,...
2124,150,614.648333
2125,400,465.269408
2126,300,340.596508
2127,500,501.629778


In [57]:
# Saving the Model 
from sklearn.externals import joblib
joblib.dump(regressor,'Food_Price_Model.obj')




['Food_Price_Model.obj']

# Conclusion
    1) One has to predict cost of the Food
       .Linear Regression
       .Handling Null Values
       .Apply the LabelEncoder
       . Prepare Linear Model
   